In [1]:
import pandas as pd

## Loading the Data

Downloading the [population estimate](https://datacatalog.worldbank.org/dataset/population-estimates-and-projections) data set from the world bank we get a zip archive containing a set of csv files. The main data is in the EstimatesData file. We also load the EstimatesCountry file containing information about countries and enabling uns to filter for "real" countries as opposed to groups of countries.

In [2]:
df = pd.read_csv("Population-EstimatesData.csv")
df_back = df

df_countries = pd.read_csv("Population-EstimatesCountry.csv")
df_countries_back = df_countries

## Understanding the Structure

Looking at the basic data structure we see a somewhat unusual way of storing the data. World Bank models the data with the time dimension as columns. This is possible since we are dealing with yearly data here, but still unusual.

Besides time the data has two more dimensions: Country and Indicator. Both are stored both as decscriptive text and as a code. I do not know what purpose the "Unnamed: 95" column has. It only contains NaN values, so we are just gonna drop it. We are also gonna drop the "Indicator Code" since it is only helpful if you have a good grasp of the World Bank indicators.

In [3]:
df.columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022',
       '2023', '2024', '2025', '2026', '2027', '2028', '2029', '2030', '2031',
       '2032', '2033', '2034', '2035', '2036', '2037', '2038', '2039', '2040',
       '2041', '2042', '2043', '2044', '2045', '2046', '2047', '2048', '2049',
       '2050', 'Unnamed: 95'],
      dtype='object')

There are 185 different indicators for all the 259 "countries". To make handling the data set a bit easies we will drop some of them. The data set seems to be tuned to enable easy visualisation of relevant data, so a lot of derived indicators are there.

In [19]:


df["Indicator Name"].value_counts()

Population ages 65 and above, total                                    259
Population ages 05-09, male (% of male population)                     259
Population ages 20-24, male                                            259
Population ages 65 and above, male (% of male population)              259
Population growth (annual %)                                           259
                                                                      ... 
Probability of dying among adolescents ages 10-14 years (per 1,000)    259
Age population, age 05, male, interpolated                             259
Age population, age 09, male, interpolated                             259
Population ages 10-14, male (% of male population)                     259
Age population, age 18, female, interpolated                           259
Name: Indicator Name, Length: 185, dtype: int64

## Filtering the Data

We drop "imterpolated" indicators which are calculated from other values in the data set. We also drop percentage or rate based indicators since we want to look at actual populations numbers. Not that depending on your use case these indicators might be useful.

In [24]:
df = df[~df["Indicator Name"].str.contains("[Ii]nterpolated")]
df = df[~df["Indicator Name"].str.contains("%")]
df = df[~df["Indicator Name"].str.contains("[Rr]ate")]
df = df[~df["Indicator Name"].str.contains("[Pp]robability")]
df["Indicator Name"].value_counts()

Population ages 65 and above, total    259
Number of under-five deaths, female    259
Population ages 10-14, female          259
Number of under-five deaths, male      259
Population ages 20-24, female          259
                                      ... 
Number of deaths ages 20-24 years      259
Population ages 45-49, female          259
Population ages 50-54, female          259
Population ages 15-19, female          259
Population ages 00-04, female          259
Name: Indicator Name, Length: 66, dtype: int64

We also filter for "real" countries. Note that the definition of a country is not as intuitive as it seems at first. You can ask [Google](https://www.google.com/search?q=how+many+countries+in+the+world) about it, but even there is no definite answer. Taiwan is one of the better known countries with a complicate answer.

If we look at the countries data set we see that we get quite a bit information about the countries. We assume that a country must have a currency to filter out groupings of countries. Looking the entries without a currency this seems to make sense.

In [26]:
df_countries[df_countries["Currency Unit"].isna()]

,Country Code,Short Name,Table Name,Long Name,2-alpha code,Currency Unit,Special Notes,Region,Income Group,WB-2 code,...,Government Accounting concept,IMF data dissemination standard,Latest population census,Latest household survey,Source of most recent Income and expenditure data,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data,Unnamed: 30
5,ARB,Arab World,Arab World,Arab World,1A,NaN,Arab World aggregate. Arab World is composed o...,NaN,NaN,1A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016.0,NaN
34,CEB,Central Europe and the Baltics,Central Europe and the Baltics,Central Europe and the Baltics,B8,NaN,Central Europe and the Baltics aggregate.,NaN,NaN,B8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016.0,NaN
47,CSS,Caribbean small states,Caribbean small states,Caribbean small states,S3,NaN,NaN,NaN,NaN,S3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016.0,NaN
59,EAP,East Asia & Pacific (excluding high income),East Asia & Pacific (excluding high income),East Asia & Pacific (excluding high income),4E,NaN,East Asia and Pacific regional aggregate (does...,NaN,NaN,4E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016.0,NaN
60,EAR,Early-demographic dividend,Early-demographic dividend,Early-demographic dividend,V2,NaN,Early-dividend countries are mostly lower-midd...,NaN,NaN,V2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,TMN,Middle East & North Africa (IDA & IBRD),Middle East & North Africa (IDA & IBRD),Middle East & North Africa (IDA & IBRD),T3,NaN,Middle East & North Africa (IDA & IBRD countri...,NaN,NaN,T3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016.0,NaN
232,TSA,South Asia (IDA & IBRD),South Asia (IDA & IBRD),South Asia (IDA & IBRD),T5,NaN,South Asia (IDA & IBRD countries) aggregate.,NaN,NaN,T5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016.0,NaN
233,TSS,Sub-Saharan Africa (IDA & IBRD),Sub-Saharan Africa (IDA & IBRD),Sub-Saharan Africa (IDA & IBRD),T6,NaN,Sub-Saharan Africa (IDA & IBRD countries) aggr...,NaN,NaN,T6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016.0,NaN
241,UMC,Upper middle income,Upper middle income,Upper middle income,XT,NaN,Upper middle income group aggregate. Upper-mid...,NaN,NaN,XT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016.0,NaN


Applying this filter we get 217 countries which seems quite a lot compared to the country numbers we get from Google. We are not gonna investigate further, for our purpose it will do.

In [30]:
country_list = df_countries[~df_countries["Currency Unit"].isna()]["Country Code"].values

df = df[df["Country Code"].isin(country_list)]

df["Country Code"].nunique()

217

## Transforming the Data

Now we get into the parts where Pandas really shines. The steps before could have been easily done with SQL or Excel. But we want to transform to a more common structure for analysis. First we are finally gonna drop some columns here because they would make the statements a bit longer and we don't need them after all.

What we want to achieve is that the year is not a column in our data structure but a value since it is one obvious dimenstion for useful visualisations. For this we have to pivot the table and Pandas offers the powerful melt function to do this.

For the pivot we specify a key identifying our measurements and a future column name where the remaining columns will be stored as values and their values the old structure in an additional column called "value". Sounds complicated? This operation is easier done than said for once, so we just try it. It is called melt in Pandas.

Since for quite a few combinations of country, indicator and year the World Bank has no information this is also a good occasion to drop these values. Among the reasons for that are that the countries did not exist at that time or they could not provide numbers at the required level of detail.

In [31]:
df = df.drop("Indicator Code", axis=1)
df = df.drop('Unnamed: 95', axis=1)

df = df.melt(id_vars = ['Country Name', 'Country Code', 'Indicator Name'], var_name = "Year")
df = df.dropna()

df

,Country Name,Country Code,Indicator Name,Year,value
0,Afghanistan,AFG,"Age dependency ratio, old",1960,5.082210e+00
1,Afghanistan,AFG,"Age dependency ratio, young",1960,7.653507e+01
2,Afghanistan,AFG,"Life expectancy at birth, female (years)",1960,3.331400e+01
3,Afghanistan,AFG,"Life expectancy at birth, male (years)",1960,3.171800e+01
4,Afghanistan,AFG,"Life expectancy at birth, total (years)",1960,3.244600e+01
...,...,...,...,...,...
1303296,Zimbabwe,ZWE,"Population, female",2050,1.227400e+07
1303297,Zimbabwe,ZWE,"Population, male",2050,1.167400e+07
1303298,Zimbabwe,ZWE,"Population, total",2050,2.394800e+07
1303299,Zimbabwe,ZWE,Rural population,2050,1.294500e+07


Now this structure is nicely normalized and does not contain unnecessary information. We could base further analysis on that. However, most tools for analysis and visualisation expect a different structure. We define a measurement as an observation of a given country in a given year. All the variables we measure commonly are stored in columns.

In order to get there we have to transform our data again. This time we want to transform the indicators from rows to columns. The goal is to have a set of indicator columns in each row for a given country and year. There is a powerful Pandas function for that called "pivot_table".

Again we must give an indentifier, in this case called "index". We have to remember that we have kept two columns for the country, its name and its code, so we have to give both. The other part is year obviously. Now the values which are to be transformed into columns have to be given as "columns". We want the indicator names as the new columns. The values for these new columns we get from the column called value in the current structure.

As before it is more difficult to explain it well than actually doing it and looking at the result. Normally we would get a nested index in Pandas as a result. This may be useful in specific use cases for easier aggregations. We just want a simple and flat structure so we reset the index after the operation. Note that we are introducing NaN values again as not all indicators are available in a given year for all countries.

In [32]:

df = df.pivot_table(index=["Country Name", "Country Code", "Year"], columns="Indicator Name", values="value")
df.reset_index(inplace=True)
df

Indicator Name,Country Name,Country Code,Year,"Age dependency ratio, old","Age dependency ratio, young","Life expectancy at birth, female (years)","Life expectancy at birth, male (years)","Life expectancy at birth, total (years)",Net migration,Number of deaths ages 10-14 years,...,"Population ages 75-79, female","Population ages 75-79, male","Population ages 80 and above, female","Population ages 80 and above, male","Population, female","Population, male","Population, total",Rural population,Sex ratio at birth (male births per female births),Urban population
0,Afghanistan,AFG,1960,5.082210,76.535066,33.314,31.718,32.446,NaN,NaN,...,16990.0,19683.0,7486.0,8294.0,4347397.0,4649576.0,8996973.0,8241137.0,NaN,755836.0
1,Afghanistan,AFG,1961,5.130137,77.558548,33.840,32.224,32.962,NaN,NaN,...,17750.0,20500.0,8358.0,9283.0,4439158.0,4730252.0,9169410.0,8373138.0,NaN,796272.0
2,Afghanistan,AFG,1962,5.139642,78.149745,34.359,32.724,33.471,-20000.0,NaN,...,18364.0,21050.0,9041.0,10013.0,4535392.0,4816049.0,9351441.0,8512056.0,1.060,839385.0
3,Afghanistan,AFG,1963,5.115523,78.488542,34.866,33.216,33.971,NaN,NaN,...,18819.0,21331.0,9456.0,10376.0,4636172.0,4907033.0,9543205.0,8657977.0,NaN,885228.0
4,Afghanistan,AFG,1964,5.063745,78.790092,35.364,33.700,34.463,NaN,NaN,...,19098.0,21378.0,9513.0,10269.0,4741531.0,5003250.0,9744781.0,8810646.0,NaN,934135.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19521,Zimbabwe,ZWE,2046,7.477791,46.917619,NaN,NaN,NaN,NaN,NaN,...,107000.0,66000.0,89000.0,41000.0,11689000.0,11083000.0,22772000.0,12912000.0,NaN,9860000.0
19522,Zimbabwe,ZWE,2047,7.687570,46.327344,NaN,NaN,NaN,-50000.0,NaN,...,111000.0,69000.0,93000.0,44000.0,11839000.0,11236000.0,23075000.0,12932000.0,1.021,10143000.0
19523,Zimbabwe,ZWE,2048,7.885619,45.691576,NaN,NaN,NaN,NaN,NaN,...,117000.0,71000.0,95000.0,46000.0,11987000.0,11385000.0,23372000.0,12944000.0,NaN,10428000.0
19524,Zimbabwe,ZWE,2049,8.076687,45.014025,NaN,NaN,NaN,NaN,NaN,...,123000.0,75000.0,97000.0,47000.0,12132000.0,11531000.0,23663000.0,12948000.0,NaN,10715000.0


## Storing the Result

For further processing we store the result as a CSV file. After resetting the index we just have an artificial counter as index which isn't helpful, so we store the CSV without the index. The resulting structure is well suited for most visualisation tools. Obviously we can also do further analysis on the data frame with Pandas in Python.

In [34]:
df.to_csv("population_pivot.csv", index=False)